In [22]:
import spotipy
import song_getter as sg
import pandas as pd
import json
import os

In [2]:
#Creating a connection to the API
#A Spotify account is required for this in order to obtain a client_id and client_secret
#These can be obtained by accessing https://developer.spotify.com/dashboard

#They should be stored as environment variables SPOTIFY_CLIENT_ID and SPOTIFY_CLIENT_SECRET 
#prior to running the code in this notebook

client_id = os.getenv("SPOTIFY_CLIENT_ID")
client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")

from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id=client_id,
                                                      client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
#Reading the scraped Eurovision song data 

eurovision = pd.read_csv("data/eurovision_per_year.csv")

In [112]:
eurovision.head()

,R/O,Year,Host_City,Country,Contestant,Song,Points,Place
0,1,2019,Tel-Aviv,Malta,Michela,Chameleon,107,14th
1,2,2019,Tel-Aviv,Albania,Jonida Maliqi,Ktheju tokës,90,17th
2,3,2019,Tel-Aviv,Czech Republic,Lake Malawi,Friend of a Friend,157,11th
3,4,2019,Tel-Aviv,Germany,S!sters,Sister,24,25th
4,5,2019,Tel-Aviv,Russia,Sergey Lazarev,Scream,370,3rd


In [ ]:
#Retrieving the track information for each song from the Spotify API, including track URI

results = eurovision["Song"].apply(lambda x: sg.df_from_search(sp, x))

In [110]:
results.head()

0                          track_name          arti...
1                                              trac...
2                                               tra...
3                                               tra...
4                       track_name               ar...
Name: Song, dtype: object

In [111]:
results[0]

,track_name,artist_name,uri
0,Chameleon,Herbie Hancock,4Ce66JznW8QbeyTdSzdGwR
1,Chameleon,Mako,0eWOYqgysbSfsiCKl02xVN
2,Chameleon,PNAU,6400BZ7REh8DshcZKU7Pfp
3,Chameleon,Emancipator,0Ddpe8znDaOuERlgLhJLWX
4,Chameleon World,Jerry Paper,643sJJ4UvTQ3DWgPDa4JiQ
5,Chameleon,Boris Brejcha,6aJuHvTP9jj8CdwfHYtYWS
6,Chameleon Skin,The Flatliners,1yof4dWVrroHKfNbTWYPDS
7,Chameleon Boy,Blue October,4k67Lnw54kvIPvHjMbfziv
8,Chameleon,Mako,6FV0HKuLea5YlbU2opCWQP
9,Chameleon Paint,Tropical Fuck Storm,70jPS3Pe6lWAmOFXR7TeV0


In [ ]:
#Saving a dataframe with several nested dataframes to a csv was too messy, so we chose to save the search results
#as a separate file. The serach results were saved as a single concatenated dataframe.

#The ID in the ID column matches the row index in the eurivision dataframe and allows dividing the concatenated 
#dataframe into the search results for individual songs.

#A row with NAs was added where the search yielded no results, in order to keep the IDs consistent.

empty_df = pd.DataFrame()
empty_df["track_name"] = [pd.NA]
empty_df["artist_name"] = [pd.NA]
empty_df["uri"] = [pd.NA]
empty_df

list_of_dfs = []
for i, j in enumerate(results):
    if pd.notna(search_results_with_uri["uri"][i]):
        list_of_dfs.append(j.assign(id=i))
    else:
        list_of_dfs.append(empty_df)

search_results_concat = pd.concat(list_of_dfs)

search_results_concat.to_csv("data/search_results_concat.csv")

In [114]:
search_results_concat.head(25)

,track_name,artist_name,uri,id
0,Chameleon,Herbie Hancock,4Ce66JznW8QbeyTdSzdGwR,0.0
1,Chameleon,Mako,0eWOYqgysbSfsiCKl02xVN,0.0
2,Chameleon,PNAU,6400BZ7REh8DshcZKU7Pfp,0.0
3,Chameleon,Emancipator,0Ddpe8znDaOuERlgLhJLWX,0.0
4,Chameleon World,Jerry Paper,643sJJ4UvTQ3DWgPDa4JiQ,0.0
5,Chameleon,Boris Brejcha,6aJuHvTP9jj8CdwfHYtYWS,0.0
6,Chameleon Skin,The Flatliners,1yof4dWVrroHKfNbTWYPDS,0.0
7,Chameleon Boy,Blue October,4k67Lnw54kvIPvHjMbfziv,0.0
8,Chameleon,Mako,6FV0HKuLea5YlbU2opCWQP,0.0
9,Chameleon Paint,Tropical Fuck Storm,70jPS3Pe6lWAmOFXR7TeV0,0.0


In [5]:
#Adding the results list as a new column 

query_search_results = eurovision.assign(query_search_results = results)

In [7]:
#Creating a new series with the right track URI for each song

search_results_with_uri = query_search_results.apply(lambda x: sg.find_right_track(x["query_search_results"] , x["Contestant"]), axis=1)

In [ ]:
#Adding that series as a new column to the dataframe

search_results_with_uri = query_search_results.assign(uri = search_results_with_uri)

In [ ]:
#Dropping the column with search results and saving it to a csv file

eurovision_with_uris = search_results_with_uri.drop("query_search_results", axis="columns")

eurovision_with_uris.to_csv("data/eurovision_with_uris.csv")

In [118]:
eurovision_with_uris.head()

,R/O,Year,Host_City,Country,Contestant,Song,Points,Place,uri
0,1,2019,Tel-Aviv,Malta,Michela,Chameleon,107,14th,51FonRnKwkcXS6e0qwpEeY
1,2,2019,Tel-Aviv,Albania,Jonida Maliqi,Ktheju tokës,90,17th,7p83db7oriaCul8WzoLr6f
2,3,2019,Tel-Aviv,Czech Republic,Lake Malawi,Friend of a Friend,157,11th,2U7Zc6NrWQgUWdBAIFrzdI
3,4,2019,Tel-Aviv,Germany,S!sters,Sister,24,25th,<NA>
4,5,2019,Tel-Aviv,Russia,Sergey Lazarev,Scream,370,3rd,<NA>


In [123]:
print(f"We retrieved URIs for {sum(eurovision_with_uris['uri'].notna())} songs out of a total of \
{len(eurovision_with_uris['uri'])}")

We retrieved URIs for 431 songs out of a total of 1322
